In [ ]:
!pip install transformers

In [ ]:
import torch
import transformers as ppb
import pandas as pd
import numpy as np

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [56]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'cse6242project-417410'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://sdk.cloud.google.com | bash

In [57]:
bucket_name = 'team054'
# Download the file from the given Google Cloud Storage bucket.
!gsutil cp gs://team054/books_enriched.csv books_enriched.csv

Copying gs://team054/books_enriched.csv...
-
Operation completed over 1 objects/13.0 MiB.                                     


In [ ]:
import pandas as pd
from ast import literal_eval
#books_df = pd.read_csv('/content/books_enriched.csv')#---if uploading through the bucket way,then uncomment this

books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/books_enriched.csv')

#books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/combined_processed_files.txt')  --- Terry's file


books_df = books_df[:1000]

In [ ]:
books_df['short_description'] = books_df.description.str.slice(0,512)

In [ ]:


df = books_df.short_description

In [ ]:
books_df2 = pd.read_table('/content/drive/MyDrive/CSE6242_project/gutenberg_books.txt', index_col=[0])

In [ ]:
books_df2['short_description'] = books_df.description

In [ ]:
# add all rows of books_df2['short_description'] to df

df = pd.concat([df, books_df2['short_description']])

In [ ]:
tokenized = df.apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

# Embed Text

In [ ]:
#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#PYTORCH_NO_CUDA_MEMORY_CACHING=1
##set CUDA_VISIBLE_DEVICES=""

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)
model = model.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [15]:
embeddings = last_hidden_states[0].cpu().data[:,0,:].numpy()

In [16]:
df.shape, embeddings.shape

((42723,), (42723, 768))

# Save Embeddings

In [17]:
embeddings_pd_df = pd.DataFrame({'book_id': range(df.shape[0])})
embeddings_pd_df['title'] = books_df['title']
embeddings_pd_df['embedding'] = embeddings.tolist()

In [18]:

embeddings_pd_df['embedding'] = embeddings_pd_df['embedding'].apply(lambda x: np.array(x))


In [19]:
embeddings_pd_json= embeddings_pd_df.to_json()

In [22]:
#from google.colab import drive
#drive.mount('/content/drive')
df.to_json('/content/drive/MyDrive/CSE6242_project/embeddings_pd_json.json')

In [ ]:
#!gsutil cp embeddings_pd_txt gs://team054

In [ ]:
#import datalab.storage as gcs
#gcs.Bucket('team054').item('to/embeddings_pd_json.json').write_to(simple_dataframe.to_json(),'text/json')

In [ ]:
#from google.cloud import storage

#def write_to_cloud(buffer):
 #   client = storage.Client()
  #  bucket = client.get_bucket('team054')
  #  blob = bucket.blob('gs://team054/embeddings_pd_json.json')
 #   blob.upload_from_file(buffer)

In [ ]:
#from google.cloud import storage

#def create_newfolder(bucket_name, destination_folder_name):
 #   storage_client = storage.Client()
 #   bucket = storage_client.get_bucket('team054')
  #  blob = bucket.blob('output')

  #  blob.upload_from_file('')

   # print('Created {} .'.format(destination_folder_name))

In [23]:
embeddings_pd_df.head()

,book_id,title,embedding
0,0,"The Hunger Games (The Hunger Games, #1)","[0.1194697842001915, 0.057542070746421814, 0.3..."
1,1,Harry Potter and the Sorcerer's Stone (Harry P...,"[-0.374781996011734, -0.272121787071228, 0.131..."
2,2,"Twilight (Twilight, #1)","[0.12502777576446533, -0.7245572805404663, -0...."
3,3,To Kill a Mockingbird,"[-0.43787628412246704, -0.29153966903686523, -..."
4,4,The Great Gatsby,"[-0.40598979592323303, -0.47040653228759766, -..."


# Train KNN

In [24]:
# from sklearn.neighbors import NearestNeighbors

#X = np.array(embeddings_pd_df['embedding'].values.tolist())
#k = 5
#nn_model = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)


In [25]:
# cell edited by Tyler (3/24/2024)

from sklearn.neighbors import NearestNeighbors

# for reproducible results
np.random.seed(1)

# 80/20 train-test split
embedding_vals = np.array(embeddings_pd_df['embedding'].values.tolist())
X_train_inds = np.random.choice(embedding_vals.shape[0], size=int(0.8*embedding_vals.shape[0]), replace=False)
X_test_inds = np.setdiff1d(np.arange(embedding_vals.shape[0]), X_train_inds)

# fit knn model
X_train = embedding_vals[X_train_inds]
k = 5
nn_model = NearestNeighbors(n_neighbors=k, algorithm="ball_tree").fit(X_train)

In [26]:
# save the trained model to mydrive as pickle file
import cloudpickle

with open('/content/drive/MyDrive/CSE6242_project/knn_model.pkl', 'wb') as f:
  cloudpickle.dump(nn_model, f)

# Query KNN

In [28]:
query = np.array([X_train[i] + np.random.normal(loc=0.0, scale=1.0, size=X_train[0].shape) for i in range(5)])

In [31]:
distances, indices = nn_model.kneighbors(query)

**Visualize Five Training Queries**

In [32]:
# cell edited by Tyler (4/8/2024)

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# initialize knowledge graph
G = nx.Graph()


i = 0
for book_rec in indices:
    i += 1
    j = 0

    # each query is a node
    G.add_node(f"Query {i}", color='skyblue')

    for index in book_rec:
        j += 1

        # color coordinate based on average rating of recommendation
        if books_df.iloc[index]['average_rating'] >= 4.5:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color='green')
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")
        elif books_df.iloc[index]['average_rating'] >= 4.0 and books_df.iloc[index]['average_rating'] < 4.5:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color='#90EE90')
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")
        elif books_df.iloc[index]['average_rating'] >= 3.0 and books_df.iloc[index]['average_rating'] < 4.0:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color='yellow')
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")
        else:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color="red")
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")

# extract node colors
node_colors = [G.nodes[node]["color"] for node in G.nodes()]

# draw knowledge graph of test query results
nx.draw(G, with_labels=True, font_size=4, node_size=300, node_color=node_colors, edge_color='black')

plt.show()

IndexError: single positional indexer is out-of-bounds

KNN Model Evaluation


Section authored by Tyler (4/8/2024)**bold text**

Test KNN **Model**

In [33]:
# for reproducible results
np.random.seed(1)

# test knn model
X_test = embedding_vals[X_test_inds]
test_query = np.array([X_test[i] + np.random.normal(loc=0.0, scale=1.0, size=X_test[0].shape) for i in range(X_test.shape[0])])
test_distances, test_indices = nn_model.kneighbors(test_query)

Visualize Test Queries

In [34]:
# initialize knowledge graph
G = nx.Graph()

# generate recommendations from test query
i = 0
avg_ratings = list()

for book_rec in test_indices:
    i += 1
    j = 0
    query_avgs = list()

    # each query is a node
    G.add_node(f"Query {i}", color='skyblue')

    for index in book_rec:
        j += 1

        # color coordinate based on average rating of recommendation
        if books_df.iloc[index]['average_rating'] >= 4.5:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color='green')
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")
        elif books_df.iloc[index]['average_rating'] >= 4.0 and books_df.iloc[index]['average_rating'] < 4.5:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color='#90EE90')
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")
        elif books_df.iloc[index]['average_rating'] >= 3.0 and books_df.iloc[index]['average_rating'] < 4.0:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color='yellow')
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")
        else:
            # each recommendation is a node
            G.add_node(f"{j}. {embeddings_pd_df.iloc[index]['title']}", color="red")
            # each recommendation will have an edge connected with its associated query
            G.add_edge(f"{j}. {embeddings_pd_df.iloc[index]['title']}", f"Query {i}")

        query_avgs.append(books_df.iloc[index]['average_rating'])

    avg_ratings.append(query_avgs)

# average rating of each recommendation for each query
avg_ratings = np.array(avg_ratings)

# spread out the nodes
position = nx.spring_layout(G, k=0.001, iterations=500)

# extract node colors
node_colors = [G.nodes[node]["color"] for node in G.nodes()]

# draw knowledge graph of test query results
nx.draw(G, position, with_labels=True, font_size=4, node_size=300, node_color=node_colors, edge_color='black')

plt.show()

IndexError: single positional indexer is out-of-bounds

**Evaluation Metrics From Test** **Queries**

All evaluation metrics used are based on relevance of each book recommendation. Relevance will be ordered by the average rating of each book  r¯i . Relevance  reli  is the ranking of each book in the recommendation list (in our case,  max(reli)=4  and  min(reli)=0 ). In cases of simply classifying book recommendations as relevant vs. irrelevant (for MAP@k and MRR), we classify a book recommendation with an average rating greater than or equal to 4.00 relevant, and otherwise irrelevant. We evaluate our BERT-embedded model with the rank-based evaluation metrics listed below:

Normalized Discounted Cumulative Gain (nDCG@k): A measure of how perfectly sorted the top k book recommendations are by relevance. A nDCG@k of 0 is an imperfectly sorted recommendation list, and a nDCG@k of 1 is a perfectly sorted recommendation list.
nDCG@k=∑ki=12reli−1log2(i+1)∑ki=12k−i−1log2(i+1)

This will be calculated for each query and then we will take the mean of each nDCG@k.

Mean Average Precision (MAP@k): First, for each of the top  k  recommendations per query,
precision@k=Number of relevant recommendations in the queryk.

Then, we obtain the average precision (AP@k) for the number of relevant recommendations  r  and each of the top  k  recommendations per query,

AP@k=1r∑i=1kprecision@i×Ri

where

Ri={10if recommendation i is relevantif recommendation i is irrelevant.

Now, for the matrix of queries  Q , each row vector of queries  q , and top  k  book recommendations per query,

MAP@k=1|Q|∑q∈QAPq@k.

Mean Reciprocal Rank (MRR): We define the reciprocal rank for each query  q  as
RR=1rank of the first relevant recommendation.

And thus for the matrix of queries  Q , we have

MRR=1|Q|∑q∈QRRq.

Evaluation Metrics From Test Queries
All evaluation metrics used are based on relevance of each book recommendation. Relevance will be ordered by the average rating of each book  r¯i . Relevance  reli  is the ranking of each book in the recommendation list (in our case,  max(reli)=4  and  min(reli)=0 ). In cases of simply classifying book recommendations as relevant vs. irrelevant (for MAP@k and MRR), we classify a book recommendation with an average rating greater than or equal to 4.00 relevant, and otherwise irrelevant. We evaluate our BERT-embedded model with the rank-based evaluation metrics listed below:

Normalized Discounted Cumulative Gain (nDCG@k): A measure of how perfectly sorted the top k book recommendations are by relevance. A nDCG@k of 0 is an imperfectly sorted recommendation list, and a nDCG@k of 1 is a perfectly sorted recommendation list.
nDCG@k=∑ki=12reli−1log2(i+1)∑ki=12k−i−1log2(i+1)

This will be calculated for each query and then we will take the mean of each nDCG@k.

Mean Average Precision (MAP@k): First, for each of the top  k  recommendations per query,
precision@k=Number of relevant recommendations in the queryk.

Then, we obtain the average precision (AP@k) for the number of relevant recommendations  r  and each of the top  k  recommendations per query,

AP@k=1r∑i=1kprecision@i×Ri

where

Ri={10if recommendation i is relevantif recommendation i is irrelevant.

Now, for the matrix of queries  Q , each row vector of queries  q , and top  k  book recommendations per query,

MAP@k=1|Q|∑q∈QAPq@k.

Mean Reciprocal Rank (MRR): We define the reciprocal rank for each query  q  as
RR=1rank of the first relevant recommendation.

And thus for the matrix of queries  Q , we have

MRR=1|Q|∑q∈QRRq.

In [35]:
def ndcg(avg_ratings_matrix, k):
    '''
        This function returns the nDCG@k.
    '''

    # we start with constructing a relevance matrix
    num_queries = avg_ratings_matrix.shape[0]
    relevance_matrix = np.zeros((num_queries, k))

    for query in range(num_queries):
        ratings = avg_ratings_matrix[query,:]
        sorted_ratings = list(np.sort(ratings))
        for recommendation in range(k):
            relevance_matrix[query, recommendation] += sorted_ratings.index(ratings[recommendation])

    # ndcg@5 computation
    ndcg_arr = np.zeros(num_queries)
    for i in range(avg_ratings_matrix.shape[0]):
        rel_i = relevance_matrix[i,:]
        ndcg_arr[i] += (((2**rel_i) - 1)/(np.log2(np.arange(k)+2))).sum()/(((2**(np.sort(rel_i)[::-1])) - 1)/(np.log2(np.arange(k)+2))).sum()

    # return the mean ndcg@5
    return ndcg_arr.mean()

print(f"Mean nDCG@5: {ndcg(avg_ratings, 5)}")


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
def map_k(avg_ratings_matrix, k):
    '''
        This function returns the mean average precision.
    '''
    # relevant recommendations
    relevant = np.zeros((avg_ratings_matrix.shape[0], k))
    for i in range(avg_ratings_matrix.shape[0]):
        for j in range(k):
            if avg_ratings_matrix[i, j] >= 4.00:
                relevant[i, j] += 1

    # avg precision per query
    precision = np.zeros((avg_ratings_matrix.shape[0], k))
    for i in range(avg_ratings_matrix.shape[0]):
        for j in range(k):
            if relevant[i, j] > 0:
                precision[i, j] += relevant[i, :j].sum()/(j+1)
    ap = np.array([precision[i, :].sum()/relevant[i, :].sum() if relevant[i, :].sum() > 0 else 0 for i in range(avg_ratings_matrix.shape[0])])

    # return the mean AP@k (MAP@k)
    return ap.mean()

print(f"MAP@k: {map_k(avg_ratings, 5)}")

In [ ]:
def mrr(avg_ratings_matrix, k):
    '''
        This function returns the mean reciprocal rank.
    '''
    recip_rank_arr = list()

    # reciprocal rank for each query
    for i in range(avg_ratings_matrix.shape[0]):
        for j in range(k):
            if avg_ratings_matrix[i, j] >= 4.00:
                recip_rank_arr.append(1/(j+1))
                break


    return np.array(recip_rank_arr).mean()

print(f"MRR: {mrr(avg_ratings, 5)}")

The BERT-embedded model sorts recommendations quite well with both nDCG@k and MRR values greater than 0.71. But the MAP@k is a little low with a value of approximately 0.259. This suggests that our BERT-embedded model returns recommendation lists that are mostly sorted by relevance, but the recommendations themselves could be better.

